<h1 style="text-align: center; font-size: 50px;">DreamBooth on Stable Diffusion 2.1</h1>

DreamBooth is a powerful technique for personalizing latent diffusion models, such as Stable Diffusion, allowing the model to be fine-tuned to generate specific images based on a limited set of data.

# Notebook Overview
- Imports
- Configurations
- Verify Assets
- Download model local
- Load the Model
- DreamBooth Training
- Inference Local Model
- Galileo Evaluate Custom metrics
- Model Service

## Imports

This command installs the "diffusers" library directly from the Hugging Face GitHub repository.The diffusers library is used to work with latent diffusion models, such as Stable Diffusion, and provides tools for image generation, fine-tuning, and other functionalities.


In [ ]:
!pip install -r ../requirements.txt --quiet

### Configuration and Secrets Loading
In this section, we load configuration parameters and API keys from separate YAML files. This separation helps maintain security by keeping sensitive information (API keys) separate from configuration settings.

- **config.yaml**: Contains non-sensitive configuration parameters like model sources and URLs
- **secrets.yaml**: Contains sensitive API keys for services like Galileo and HuggingFace


> ⚠️ **Warning**: If your GPU uses an older architecture (e.g., **pre-Pascal**, such as **Maxwell or earlier**), please uncomment the following line to avoid CUDA timeout issues:
```python
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
```


In [ ]:
import os
import sys

#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from src.utils import (
    load_config_and_secrets,
    configure_proxy,
    setup_galileo_environment,
    get_project_root,
    get_config_dir,
    get_output_dir,
    get_default_model_path,
    get_model_cache_dir
)


# Add the core directory to the path to import utils
core_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if core_path not in sys.path:
    sys.path.append(core_path)

from core.custom_metrics.image_metrics_scorers import entropy_scorer, complexity_scorer, set_custom_image_path
from core.deploy.deploy_image_generation import deploy_model
from core.local_inference.inference import StableDiffusionPipelineOutput, load_config, run_inference
from core.dreambooth_inference.inference_dreambooth import StableDiffusionPipelineOutput, load_config_dreambooth, run_inference_dreambooth


# === Third-Party Imports ===
from huggingface_hub import snapshot_download
import promptquality as pq
import glob
import torch


# === Standard Library Imports ===
import logging
import warnings
from pathlib import Path

## Configurations

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
# === Create logger ===
logger = logging.getLogger("image-generation-notebook")
logger.setLevel(logging.INFO)

formatter = logging.Formatter(
    "%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S"
)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.propagate = False

In [ ]:
# === Constants ===
# Model and experiment configuration
project_root = get_project_root()
CONFIG_PATH = str(project_root / "configs" / "config.yaml")
SECRETS_PATH = str(project_root / "configs" / "secrets.yaml")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
logger.info('Notebook execution started.')

### Configuration and Secrets Loading
In this section, we load configuration parameters and API keys from separate YAML files. This separation helps maintain security by keeping sensitive information (API keys) separate from configuration settings.

- **config.yaml**: Contains non-sensitive configuration parameters like model sources and URLs
- **secrets.yaml**: Contains sensitive API keys for services like Galileo and HuggingFace


In [ ]:
config, secrets = load_config_and_secrets(CONFIG_PATH, SECRETS_PATH)

In [ ]:
configure_proxy(config)

## Verify Assets

In [ ]:
def log_asset_status(asset_path: str, asset_name: str, success_message: str, failure_message: str) -> None:
    """
    Logs the status of a given asset based on its existence.

    Parameters:
        asset_path (str): File or directory path to check.
        asset_name (str): Name of the asset for logging context.
        success_message (str): Message to log if asset exists.
        failure_message (str): Message to log if asset does not exist.
    """
    if Path(asset_path).exists():
        logger.info(f"{asset_name} is properly configured. {success_message}")
    else:
        logger.info(f"{asset_name} is not properly configured. {failure_message}")


log_asset_status(
    asset_path=CONFIG_PATH,
    asset_name="Config",
    success_message="",
    failure_message="Please check if the configs.yaml was propely connfigured in your project on AI Studio."
)

log_asset_status(
    asset_path=SECRETS_PATH,
    asset_name="Secrets",
    success_message="",
    failure_message="Please check if the secrets.yaml was propely connfigured in your project on AI Studio."
)

## Download model local

This code imports the snapshot_download function from the huggingface_hub library to download the latest version of the "stabilityai/stable-diffusion-2-1" model. It sets a local directory for saving the model (local_model_path), and the download is configured to be resumable in case it is interrupted, with an etag timeout set to 60 seconds.

In [ ]:
# Download the snapshot directly to the project's models directory
local_model_path = str(get_project_root() / "models" / "stable-diffusion-2-1")

# Downloading the latest revision of the "stabilityai/stable-diffusion-2-1" model
snapshot_download(
    repo_id="stabilityai/stable-diffusion-2-1", 
    local_dir=local_model_path,
    resume_download=True,
    etag_timeout=60  
)

## Load the Model
We load the pre-trained Stable Diffusion 2.1 model from Hugging Face and move it to the GPU for efficient execution

In [ ]:
config = load_config()

run_inference(
    prompt="A beautiful landscape",
    height=768,
    width=768,
    num_images=1,
    num_inference_steps=60,
    output=True  
)



## DreamBooth Training

This Bash script uses PyTorch to detect the number of available GPUs, automatically selects either the multi-GPU or single-GPU configuration file, and then launches the DreamBooth training process on Stable Diffusion via `accelerate launch` with your specified parameters. It also captures the start and end timestamps to calculate the total training duration.

> **Tip:** To enable full, verbose logging, add the `--log_level=DEBUG` flag to your command.


In [ ]:
%%bash -s "${{str(project_root)}}"
# Use the project_root variable from Python converted to string
PROJECT_ROOT="$1"

CONFIG_DIR="$PROJECT_ROOT/config"
OUTPUT_DIR="$PROJECT_ROOT/output"
DATA_DIR="$PROJECT_ROOT/data"

# Ensure directories exist
mkdir -p "$OUTPUT_DIR"
mkdir -p "$DATA_DIR"

NUM_GPUS=$(python3 -c "import torch; print(torch.cuda.device_count())")

if [ "$NUM_GPUS" -ge 2 ]; then
  CONFIG_FILE="$CONFIG_DIR/default_config_multi-gpu.yaml"
  echo "Detected $NUM_GPUS GPUs, using $CONFIG_FILE"
else
  CONFIG_FILE="$CONFIG_DIR/default_config_one-gpu.yaml"
  echo "Detected $NUM_GPUS GPU, using $CONFIG_FILE"
fi

# Verify config file exists
if [ ! -f "$CONFIG_FILE" ]; then
  echo "Error: Config file $CONFIG_FILE not found!"
  exit 1
fi

START=$(date +%s)

accelerate launch --config_file "$CONFIG_FILE" ../core/train/train_dreambooth_aistudio.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-2-1"  \
  --log_level=ERROR \
  --instance_data_dir="$DATA_DIR/img" \
  --output_dir="$OUTPUT_DIR/dreambooth/" \
  --instance_prompt="A modern laptop on a sandy beach with the ocean in the background, sunlight reflecting off the screen" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=400 \
  --logging_dir="/phoenix/tensorboard/tensorlogs" \
  --report_to="tensorboard" \
  --validation_prompt="A photo of an HP laptop on the sand with a sunset over the ocean in the background." \
  --num_validation_images=1 \
  --validation_steps=100

END=$(date +%s)
DIFF=$(( $END - $START ))

echo "Training completed in $DIFF seconds"

## Inference Local Model

This code imports functions from the inference_dreambooth module, loads a configuration, and then runs inference to generate images. It uses a prompt to create three images with a resolution of 768x768 pixels, executing 100 inference steps per image.

In [ ]:
config = load_config_dreambooth()

run_inference_dreambooth(
    prompt="A high-quality photo of an HP laptop placed on the sand at the beach, with a sunset over the ocean in the background.", 
    height=768, 
    width=768, 
    num_images=3, 
    num_inference_steps=100,
    output=True
)


## Galileo Evaluate Custom metrics
Galileo GenAI Studio supports Custom Metrics (programmatic or GPT-based) for all your Evaluate and Observe projects. 

In [ ]:
#########################################
# In order to connect to Galileo, create a secrets.yaml file in the configs folder.
# This file should be an entry called GALILEO_API_KEY, with your personal Galileo API Key
# Galileo API keys can be created on https://console.hp.galileocloud.io/settings/api-keys
#########################################

setup_galileo_environment(secrets)
pq.login(os.environ['GALILEO_CONSOLE_URL'])

In [ ]:
def get_latest_generated_image(directory: str = None, prefix: str = "dreambooth_result_", ext: str = ".png") -> str:
    """
    Returns the path of the most recent image that matches the specified pattern.
    Uses the project's output directory by default.
    """
    if directory is None:
        directory = str(get_output_dir())
    
    files = glob.glob(os.path.join(directory, f"{prefix}*{ext}"))
    if not files:
        raise FileNotFoundError(f"No generated images found in {directory}.")
    latest_file = max(files, key=os.path.getmtime)
    return latest_file

In [ ]:
config = load_config_dreambooth()

prompt_text = ("A high-quality photo of an HP laptop placed on the sand at the beach, "
               "with a sunset over the ocean in the background.")

run_inference_dreambooth(
    prompt=prompt_text, 
    height=768, 
    width=768, 
    num_images=1, 
    num_inference_steps=100,
    output=True
)

generated_image_path = get_latest_generated_image()

set_custom_image_path(generated_image_path)

template = prompt_text

result_custom = pq.run(template=template, scorers=[entropy_scorer, complexity_scorer])
print("Result:", result_custom)

## Model Service

Using MLflow, we will save and load the model in an integrated manner, enabling the traceability and reproducibility of experiments. MLflow will facilitate model versioning, monitoring, and deployment, ensuring a robust pipeline for your project.

In [ ]:
deploy_model()

In [ ]:
logger.info('Notebook execution completed.')

Built with ❤️ using Z by HP AI Studio